In [1]:
import sys
assert sys.version_info >=(3,7)

import numpy as np
import cv2 as cv

if not cv.useOptimized():
    cv.setUseOptimized(True)
    
cv.useOptimized()

True

# 1
Convert the code chunk found under section Divide an image into smaller patches using cropping into a function with the following signature:
crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color)
#img is the source image
#num_horizontal_grid and num_vertical_grid are the number of patches along x and y axes.
#line_color is the color of the grid line.
#The output of the function should be image with grids

In [3]:
def  crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    img_copy = img.copy()
    height, width = img.shape[:2]

    # M and N are basically number of pixels per patch
    M, N = height//num_horizontal_grid, width//num_vertical_grid

    x1, y1 = 0, 0

    for y in range(0, height, M):
        for x in range(0, width, N):

            y1 = y + M    # lower right coordinate to construct rectangle
            x1 = x + N

            # Check whether patch lower right coordinate exceeds image height and width
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                tile = img[y:height, x:width]
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            # When patch lower right y-coordinate exceeds patch height
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            # When patch lower right x-coordinate exceeds patch width
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
    return img_copy

img = cv.imread('lena.jfif')
num_horizontal_grid = 3
num_vertical_grid = 3
result=crop_grid(img, num_horizontal_grid, num_vertical_grid, 0) 

cv.imshow('Q1', result)
cv.waitKey(0)
cv.destroyAllWindows()

# 2
Display image sequences of smooth transition of two images with different values of 
α. Refer to code in section "Image blending". Use "lena.jfif" and "coins.jfif" as the base images.

In [4]:
#2 way = for loop, gui = track bar

img1 = cv.imread('lena.jfif')
img2 = cv.imread('coins.jfif')

# Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

a=[0,0.25,0.5,0.75,1]
for i in range (5):
    dst = cv.addWeighted(img1, a[i], img2, a[4-i], 0)
    img_name='Q2'+chr(i)
    cv.imshow(img_name, dst)
    

cv.waitKey(0)
cv.destroyAllWindows()

# 3
Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use "lena.jfif" as the input image.
Use external libraries imutils.
Modify the transformation matrix.

In [5]:
img = cv.imread('lena.jfif')
rows, cols = img.shape[:2]
centreY, centreX = rows//2, cols//2
M = cv.getRotationMatrix2D(((cols-1)/2.0, (rows-1)/2.0), 45, 1)

cosM = np.abs(M[0][0])
sinM = np.abs(M[0][1])

newHeight = int((rows * sinM) +(cols * cosM))
newWidth = int((rows * cosM)+(cols * sinM))

M[0][2] += (newWidth/2) - centreX
M[1][2] += (newHeight/2) - centreY
    
dst = cv.warpAffine(img, M, (newWidth, newHeight))

cv.imshow('Q3', dst)
cv.waitKey(0)
cv.destroyAllWindows()

# 4
Use the images with titles: "flower.jfif" and "native-bee.png". I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are cv.bitwise_and(), cv.bitwise_or() and cv.bitwise_not(). You need to use cv.threshold function to segment the flower. Please refer to online documentation for more info. The result should resemble the following:

In [6]:
img1 = cv.imread('native-bee.png')
img2 = cv.imread('flower.jfif')

#create a roi
rows, cols = img2.shape[:2]
roi = img1[0:rows, 0:cols]

#create mask of flower
img2_Gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(img2_Gray, 70, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

#blackout the area of flower
img1B = cv.bitwise_and(roi,roi,mask = mask_inv)

#take region of flower
img2F = cv.bitwise_and(img2,img2,mask = mask)


dst = cv.add(img1B,img2F)
img1[0:rows, 0:cols ] = dst


cv.imshow('Q4', img1)
cv.waitKey(0)
cv.destroyAllWindows()